In [1]:
from faker import Faker
import json
import numpy as np
import pandas as pd
import random
from datetime import date

In [2]:
random.seed(444)
Faker.seed(777)
np.random.seed(1616)

In [19]:
#Osanikud
def juhuslik_registrikood(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

def juhuslik_osaniku_kapital():
    return random.randint(2500,1e6)

def pseudo_isikukood(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

def genereeri_fuusiline_isik():
    fake=Faker(['et_EE'])
    fuusiline_isik=fake.name()
    isikukood=pseudo_isikukood(11)
    return {'nimi': fuusiline_isik,'isikukood':str(isikukood)}
    
def genereeri_juriidiline_isik():
    fake=Faker(['et_EE'])
    juriidiline_isik=fake.company()
    registrikood=juhuslik_registrikood(8)
    return {'nimi': juriidiline_isik, 'registrikood': registrikood}

def genereeri_juriidilised_isikud_tabel():
    juriidilised_isikud=[]
    for i in range(0,200):
        juriidilised_isikud.append(genereeri_juriidiline_isik())
    with open('juriidilised_isikud_test.json', 'w', encoding='utf-8') as f:
        json.dump(juriidilised_isikud,f,ensure_ascii=False)
    
def genereeri_fuusilised_isikud_tabel():
    fuusilised_isikud=[]
    for i in range(0,200):
        fuusilised_isikud.append(genereeri_fuusiline_isik())
    with open('fuusilised_isikud_test.json', 'w', encoding='utf-8') as f:
        json.dump(fuusilised_isikud,f,ensure_ascii=False)

def genereeri_osanikud():
    osanike_n=np.random.randint(1,5)
    osanikud=np.random.choice(np.arange(0,200), osanike_n, replace=False)
    return osanikud

def genereeri_assotsiatsiooni_tabel(fuusilised_isikud=True):
    assotsiatsiooni_tabel=[]
    for i in range(0,100):
        osanikud=genereeri_osanikud()
        for o in osanikud:
            assotsiatsiooni_tabel.append({'left_id_osauhingud': int(i), 'right_id_osanikud': int(o), 
                                          'osakapital': juhuslik_osaniku_kapital(),
                                         'is_asutaja':random.choice(['On', 'Ei ole'])})
    assotsiatsiooni_tabel_df=pd.DataFrame(assotsiatsiooni_tabel)
    if fuusilised_isikud:
        with open('fuusilised_isikud_assotsiatsiooni_tabel_test.json', 'w', encoding='utf-8') as f:
            json.dump(assotsiatsiooni_tabel,f,ensure_ascii=False)
    else:
        with open('juriidilised_isikud_assotsiatsiooni_tabel_test.json', 'w', encoding='utf-8') as f:
            json.dump(assotsiatsiooni_tabel,f,ensure_ascii=False)
    return assotsiatsiooni_tabel_df

assotsiatsiooni_tabel_1_df=genereeri_assotsiatsiooni_tabel()
assotsiatsiooni_tabel_2_df=genereeri_assotsiatsiooni_tabel(fuusilised_isikud=False)
    

In [11]:
#Genereeri osanikud ilma kapitalita
def juhuslik_registrikood(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

def juhusliku_osauhingu_nimi():
    fake=Faker(['et_EE'])
    osauhingu_nimi=fake.company()
    while len(osauhingu_nimi)<3 or len(osauhingu_nimi)>100:
        osauhingu_nimi=fake.company()
    return osauhingu_nimi

def juhuslik_asutamise_kuupaev():
    fake=Faker()
    juhuslik_asutamise_kuupaev=str(fake.date_between(date.fromisoformat('1986-04-16'),date.today()))
    return juhuslik_asutamise_kuupaev

def juhuslik_osauhing():
    juhuslik_osauhing = {
        'osauhingu_nimi': juhusliku_osauhingu_nimi(),
        'registri_kood': juhuslik_registrikood(7),
        'asutamise_kuupaev': juhuslik_asutamise_kuupaev()
    }
    return juhuslik_osauhing

def osauhingud_ilma_kapitalita_df():
    osauhingud_ilma_kapitalita=[]

    for a in range(0,100):
        osauhingud_ilma_kapitalita.append(juhuslik_osauhing())
        
    osauhingud_ilma_kapitalita=pd.DataFrame(osauhingud_ilma_kapitalita)
    return osauhingud_ilma_kapitalita

osauhingud_ilma_kapitalita=osauhingud_ilma_kapitalita_df()

In [12]:
osauhingud_ilma_kapitalita

,osauhingu_nimi,registri_kood,asutamise_kuupaev
0,Võsu-Lind,3715003,1998-03-26
1,Käär-Õun,9235541,2016-01-07
2,"Lember, Iljin and Kasemets",6810930,1992-08-01
3,"Fjodorov, Tomingas and Toom",1542680,1995-07-11
4,Jürgenson PLC,7972358,2007-06-10
...,...,...,...
95,"Palm, Maasik and Kärner",6092299,2011-01-31
96,Koger Inc,5605363,2017-04-06
97,"Konstantinov, Solovjov and Kiik",8957457,2016-05-21
98,"Suur, Koval and Sokk",4589589,1987-11-23


In [18]:
def total_capital_calculation(osauhingud_ilma_kapitalita, osauhing_assoc_1, osauhing_assoc_2):
    # Group association tables by "osauhing" and sum the "osamakse" column
    osauhing_assoc_1_sum = osauhing_assoc_1.groupby("left_id_osauhingud")["osakapital"].sum()
    osauhing_assoc_2_sum = osauhing_assoc_2.groupby("left_id_osauhingud")["osakapital"].sum()

    # Merge the two association tables on "osauhing"
    osauhing_assoc_merged = pd.merge(osauhing_assoc_1_sum, osauhing_assoc_2_sum, on="left_id_osauhingud", how="inner")

    # Sum the two "osamakse" columns
    osauhing_assoc_merged_sum = osauhing_assoc_merged.sum(axis=1)

    # Add the new column "osakapital" to "osauhingud_ilma_kapitalita" with the calculated values
    osauhingud_ilma_kapitalita["kapital"] = osauhing_assoc_merged_sum
    
    osauhingud_kapitaliga=osauhingud_ilma_kapitalita
    
    osauhingud_kapitaliga.to_json('test_andmestik.json')
    
    return osauhingud_kapitaliga

osauhingud_df=total_capital_calculation(osauhingud_ilma_kapitalita, assotsiatsiooni_tabel_1_df, assotsiatsiooni_tabel_2_df)

In [15]:
osauhingud_df

,osauhingu_nimi,registri_kood,asutamise_kuupaev,osakapital
0,Võsu-Lind,3715003,1998-03-26,845876
1,Käär-Õun,9235541,2016-01-07,1036958
2,"Lember, Iljin and Kasemets",6810930,1992-08-01,3100534
3,"Fjodorov, Tomingas and Toom",1542680,1995-07-11,3400660
4,Jürgenson PLC,7972358,2007-06-10,1744115
...,...,...,...,...
95,"Palm, Maasik and Kärner",6092299,2011-01-31,2220581
96,Koger Inc,5605363,2017-04-06,2305977
97,"Konstantinov, Solovjov and Kiik",8957457,2016-05-21,1136072
98,"Suur, Koval and Sokk",4589589,1987-11-23,4340183
